<style>
    rd { color:red; }
    bl { color:blue; }
</style>

# Feature Engineering 을 통한 성능 향상 시도
XGBoost의 plot_importance 결과로 얻은, 중요도가 떨어지는 속성들에 대하여<br/>
noise 일 수 도 있는 속성들을 제거하여 성능이 향상되는지 확인해본다.

## 전처리
| 작업        | 대상                                                                                       |
|:------------|:-------------------------------------------------------------------------------------------|
| 컬럼 삭제   | "RowNumber", "CustomerId", "Surname"                                                       |
| 컬럼 인코딩 | "Geography", "Gender"                                                                      |
| 컬럼 라벨링 | "CreditScore", "Geography", "Age", "Tenure", "Balance", "NumOfProducts", "EstimatedSalary" |

### 스케일링 : StandardScaler

## Drop 대상 특성
1. HasCrCard
2. CreditScore
3. EstimatedSalary
4. Tenure

## 결론 : 정확도 86% 에서 의미있는 성능변화 없음

In [20]:
import numpy             as np
import pandas            as pd
import matplotlib.pyplot as plt
import seaborn           as sns

import matplotlib
import matplotlib.font_manager as fm

import re

from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [21]:
def encoding(df:pd.DataFrame, columns:list[str]):
    """범주형 데이터를 인코딩"""

    encoder_list = {}
    result_df    = df.copy(deep=True)

    for col_nm in columns:
        encoder           = LabelEncoder()
        result_df[col_nm] = encoder.fit_transform(result_df[col_nm])

        encoder_list[col_nm] = encoder

    return result_df, encoder_list


def scaling(df:pd.DataFrame, columns:list[str]):
    """DataFrame 에서 컬럼들을 스케일링"""

    scaler    = StandardScaler()
    result_df = df.copy(deep=True)

    result_df[columns] = scaler.fit_transform(result_df[columns])

    return result_df

## 데이터 로드 및 전처리

In [22]:
######################################### 데이터 로드
df     = pd.read_csv("../data/Churn_Modelling.csv")
inputs = df.drop(columns=["Exited"], axis=1)
labels = df["Exited"]


######################################### 데이터 전처리
_input = inputs.drop(columns=["RowNumber", "CustomerId", "Surname"], axis=1)     # 컬럼 삭제( Rownumber, CustomerId, Surname )
_input, encoders = encoding(_input, ["Geography", "Gender"])            # 범주형 문자열 데이터 인코딩
_input = scaling(_input, ["CreditScore", "Geography", "Age", "Tenure", "Balance", "NumOfProducts", "EstimatedSalary"])

## TRIAL : XGBoost의 plot_importance 에서 중요도가 떨어지는 특성들을 제거해가며 성능 변화 확인
### 1. HasCrCard 속성 drop 후 성능 확인
성능이 미세하게 하락

In [23]:
######################################### DROP : HasCrCard
fe_input = _input.drop(columns=["HasCrCard"])


######################################### 데이터 분할. random_state 지정한 상태에서 성능 확인/개선해보고, state 풀었을 때도 보기.
train_x, test_x, train_y, test_y = train_test_split(fe_input, labels, stratify=labels)
# print("학습 데이터 shape : ", train_x.shape, train_y.shape)
# print("검증 데이터 shape : ",  test_x.shape,  test_y.shape, "\n")


######################################### 모델 학습
model = RandomForestClassifier(max_depth=10)
model.fit(train_x, train_y)


######################################### 모델 성능 평가
predicted = model.predict(test_x)
print(classification_report(test_y, predicted, target_names=["Stayed", "Exited"]))

              precision    recall  f1-score   support

      Stayed       0.87      0.97      0.92      1991
      Exited       0.80      0.45      0.57       509

    accuracy                           0.86      2500
   macro avg       0.83      0.71      0.75      2500
weighted avg       0.86      0.86      0.85      2500



### 2. CreditScore 속성 drop 후 성능 확인
성능이 미세하게 하락

In [24]:
######################################### DROP : CreditScore
fe_input = _input.drop(columns=["CreditScore"])


######################################### 데이터 분할. random_state 지정한 상태에서 성능 확인/개선해보고, state 풀었을 때도 보기.
train_x, test_x, train_y, test_y = train_test_split(fe_input, labels, stratify=labels)
# print("학습 데이터 shape : ", train_x.shape, train_y.shape)
# print("검증 데이터 shape : ",  test_x.shape,  test_y.shape, "\n")


######################################### 모델 학습
model = RandomForestClassifier(max_depth=10)
model.fit(train_x, train_y)


######################################### 모델 성능 평가
predicted = model.predict(test_x)
print(classification_report(test_y, predicted, target_names=["Stayed", "Exited"]))

              precision    recall  f1-score   support

      Stayed       0.86      0.97      0.92      1991
      Exited       0.80      0.39      0.53       509

    accuracy                           0.86      2500
   macro avg       0.83      0.68      0.72      2500
weighted avg       0.85      0.86      0.84      2500



In [25]:
######################################### DROP : EstimatedSalary
fe_input = _input.drop(columns=["EstimatedSalary"])


######################################### 데이터 분할. random_state 지정한 상태에서 성능 확인/개선해보고, state 풀었을 때도 보기.
train_x, test_x, train_y, test_y = train_test_split(fe_input, labels, stratify=labels)
# print("학습 데이터 shape : ", train_x.shape, train_y.shape)
# print("검증 데이터 shape : ",  test_x.shape,  test_y.shape, "\n")


######################################### 모델 학습
model = RandomForestClassifier(max_depth=10)
model.fit(train_x, train_y)


######################################### 모델 성능 평가
predicted = model.predict(test_x)
print(classification_report(test_y, predicted, target_names=["Stayed", "Exited"]))

              precision    recall  f1-score   support

      Stayed       0.87      0.98      0.92      1991
      Exited       0.82      0.42      0.56       509

    accuracy                           0.86      2500
   macro avg       0.84      0.70      0.74      2500
weighted avg       0.86      0.86      0.85      2500



In [26]:
######################################### DROP : Tenure
fe_input = _input.drop(columns=["Tenure"])


######################################### 데이터 분할. random_state 지정한 상태에서 성능 확인/개선해보고, state 풀었을 때도 보기.
train_x, test_x, train_y, test_y = train_test_split(fe_input, labels, stratify=labels)
# print("학습 데이터 shape : ", train_x.shape, train_y.shape)
# print("검증 데이터 shape : ",  test_x.shape,  test_y.shape, "\n")


######################################### 모델 학습
model = RandomForestClassifier(max_depth=10)
model.fit(train_x, train_y)


######################################### 모델 성능 평가
predicted = model.predict(test_x)
print(classification_report(test_y, predicted, target_names=["Stayed", "Exited"]))

              precision    recall  f1-score   support

      Stayed       0.87      0.97      0.91      1991
      Exited       0.77      0.43      0.55       509

    accuracy                           0.86      2500
   macro avg       0.82      0.70      0.73      2500
weighted avg       0.85      0.86      0.84      2500



In [27]:
######################################### DROP : HasCrCard, CreditScore
fe_input = _input.drop(columns=["HasCrCard", "CreditScore"])


######################################### 데이터 분할. random_state 지정한 상태에서 성능 확인/개선해보고, state 풀었을 때도 보기.
train_x, test_x, train_y, test_y = train_test_split(fe_input, labels, stratify=labels)
# print("학습 데이터 shape : ", train_x.shape, train_y.shape)
# print("검증 데이터 shape : ",  test_x.shape,  test_y.shape, "\n")


######################################### 모델 학습
model = RandomForestClassifier(max_depth=10)
model.fit(train_x, train_y)


######################################### 모델 성능 평가
predicted = model.predict(test_x)
print(classification_report(test_y, predicted, target_names=["Stayed", "Exited"]))

              precision    recall  f1-score   support

      Stayed       0.87      0.97      0.92      1991
      Exited       0.80      0.43      0.56       509

    accuracy                           0.86      2500
   macro avg       0.84      0.70      0.74      2500
weighted avg       0.86      0.86      0.84      2500



In [28]:
######################################### DROP : HasCrCard, CreditScore
fe_input = _input.drop(columns=["HasCrCard", "EstimatedSalary"])


######################################### 데이터 분할. random_state 지정한 상태에서 성능 확인/개선해보고, state 풀었을 때도 보기.
train_x, test_x, train_y, test_y = train_test_split(fe_input, labels, stratify=labels)
# print("학습 데이터 shape : ", train_x.shape, train_y.shape)
# print("검증 데이터 shape : ",  test_x.shape,  test_y.shape, "\n")


######################################### 모델 학습
model = RandomForestClassifier(max_depth=10)
model.fit(train_x, train_y)


######################################### 모델 성능 평가
predicted = model.predict(test_x)
print(classification_report(test_y, predicted, target_names=["Stayed", "Exited"]))

              precision    recall  f1-score   support

      Stayed       0.87      0.97      0.92      1991
      Exited       0.78      0.43      0.56       509

    accuracy                           0.86      2500
   macro avg       0.82      0.70      0.74      2500
weighted avg       0.85      0.86      0.84      2500



In [29]:
######################################### DROP : HasCrCard, CreditScore
fe_input = _input.drop(columns=["HasCrCard", "Tenure"])


######################################### 데이터 분할. random_state 지정한 상태에서 성능 확인/개선해보고, state 풀었을 때도 보기.
train_x, test_x, train_y, test_y = train_test_split(fe_input, labels, stratify=labels)
# print("학습 데이터 shape : ", train_x.shape, train_y.shape)
# print("검증 데이터 shape : ",  test_x.shape,  test_y.shape, "\n")


######################################### 모델 학습
model = RandomForestClassifier(max_depth=10)
model.fit(train_x, train_y)


######################################### 모델 성능 평가
predicted = model.predict(test_x)
print(classification_report(test_y, predicted, target_names=["Stayed", "Exited"]))

              precision    recall  f1-score   support

      Stayed       0.87      0.98      0.92      1991
      Exited       0.82      0.41      0.55       509

    accuracy                           0.86      2500
   macro avg       0.84      0.69      0.73      2500
weighted avg       0.86      0.86      0.84      2500



In [30]:
######################################### DROP : HasCrCard, CreditScore
fe_input = _input.drop(columns=["CreditScore", "Tenure"])


######################################### 데이터 분할. random_state 지정한 상태에서 성능 확인/개선해보고, state 풀었을 때도 보기.
train_x, test_x, train_y, test_y = train_test_split(fe_input, labels, stratify=labels)
# print("학습 데이터 shape : ", train_x.shape, train_y.shape)
# print("검증 데이터 shape : ",  test_x.shape,  test_y.shape, "\n")


######################################### 모델 학습
model = RandomForestClassifier(max_depth=10)
model.fit(train_x, train_y)


######################################### 모델 성능 평가
predicted = model.predict(test_x)
print(classification_report(test_y, predicted, target_names=["Stayed", "Exited"]))

              precision    recall  f1-score   support

      Stayed       0.86      0.98      0.92      1991
      Exited       0.81      0.40      0.54       509

    accuracy                           0.86      2500
   macro avg       0.84      0.69      0.73      2500
weighted avg       0.85      0.86      0.84      2500



In [31]:
######################################### DROP : HasCrCard, CreditScore
fe_input = _input.drop(columns=["CreditScore", "EstimatedSalary"])


######################################### 데이터 분할. random_state 지정한 상태에서 성능 확인/개선해보고, state 풀었을 때도 보기.
train_x, test_x, train_y, test_y = train_test_split(fe_input, labels, stratify=labels)
# print("학습 데이터 shape : ", train_x.shape, train_y.shape)
# print("검증 데이터 shape : ",  test_x.shape,  test_y.shape, "\n")


######################################### 모델 학습
model = RandomForestClassifier(max_depth=10)
model.fit(train_x, train_y)


######################################### 모델 성능 평가
predicted = model.predict(test_x)
print(classification_report(test_y, predicted, target_names=["Stayed", "Exited"]))

              precision    recall  f1-score   support

      Stayed       0.87      0.97      0.92      1991
      Exited       0.77      0.44      0.56       509

    accuracy                           0.86      2500
   macro avg       0.82      0.70      0.74      2500
weighted avg       0.85      0.86      0.84      2500



In [32]:
######################################### DROP : HasCrCard, CreditScore
fe_input = _input.drop(columns=["HasCrCard", "CreditScore", "EstimatedSalary"])


######################################### 데이터 분할. random_state 지정한 상태에서 성능 확인/개선해보고, state 풀었을 때도 보기.
train_x, test_x, train_y, test_y = train_test_split(fe_input, labels, stratify=labels)
# print("학습 데이터 shape : ", train_x.shape, train_y.shape)
# print("검증 데이터 shape : ",  test_x.shape,  test_y.shape, "\n")


######################################### 모델 학습
model = RandomForestClassifier(max_depth=10)
model.fit(train_x, train_y)


######################################### 모델 성능 평가
predicted = model.predict(test_x)
print(classification_report(test_y, predicted, target_names=["Stayed", "Exited"]))

              precision    recall  f1-score   support

      Stayed       0.87      0.97      0.92      1991
      Exited       0.77      0.45      0.57       509

    accuracy                           0.86      2500
   macro avg       0.82      0.71      0.74      2500
weighted avg       0.85      0.86      0.85      2500



In [33]:
######################################### DROP : HasCrCard, CreditScore
fe_input = _input.drop(columns=["HasCrCard", "CreditScore", "Tenure"])


######################################### 데이터 분할. random_state 지정한 상태에서 성능 확인/개선해보고, state 풀었을 때도 보기.
train_x, test_x, train_y, test_y = train_test_split(fe_input, labels, stratify=labels)
# print("학습 데이터 shape : ", train_x.shape, train_y.shape)
# print("검증 데이터 shape : ",  test_x.shape,  test_y.shape, "\n")


######################################### 모델 학습
model = RandomForestClassifier(max_depth=10)
model.fit(train_x, train_y)


######################################### 모델 성능 평가
predicted = model.predict(test_x)
print(classification_report(test_y, predicted, target_names=["Stayed", "Exited"]))

              precision    recall  f1-score   support

      Stayed       0.87      0.97      0.92      1991
      Exited       0.79      0.42      0.55       509

    accuracy                           0.86      2500
   macro avg       0.83      0.70      0.73      2500
weighted avg       0.85      0.86      0.84      2500



In [35]:
######################################### DROP : HasCrCard, CreditScore
fe_input = _input.drop(columns=["HasCrCard", "CreditScore", "Tenure", "EstimatedSalary"])


######################################### 데이터 분할. random_state 지정한 상태에서 성능 확인/개선해보고, state 풀었을 때도 보기.
train_x, test_x, train_y, test_y = train_test_split(fe_input, labels, stratify=labels)
# print("학습 데이터 shape : ", train_x.shape, train_y.shape)
# print("검증 데이터 shape : ",  test_x.shape,  test_y.shape, "\n")


######################################### 모델 학습
model = RandomForestClassifier(max_depth=10)
model.fit(train_x, train_y)


######################################### 모델 성능 평가
predicted = model.predict(test_x)
print(classification_report(test_y, predicted, target_names=["Stayed", "Exited"]))

              precision    recall  f1-score   support

      Stayed       0.88      0.96      0.92      1991
      Exited       0.75      0.47      0.58       509

    accuracy                           0.86      2500
   macro avg       0.81      0.72      0.75      2500
weighted avg       0.85      0.86      0.85      2500

